In [5]:
import os
from __future__ import print_function, division
import pandas as pd
from datetime import timedelta
from time import strftime
import numpy as np

path = '/Users/kang/Desktop/energydisagg/data'# multi_group
os.chdir(path)

In [6]:
"""
    prepare training data from 'multi_group', which can be downloaded from google drive
    collection : data_in_dictionary, training data
    train_builing : IdList, the building for training
    bprob = IdProbility, the probabilities for the training builings. 
    activation_prob = ActivationProbility, the probabilities for the activations
    
""" 

"\n    prepare training data from 'multi_group', which can be downloaded from google drive\n    collection : data_in_dictionary, training data\n    train_builing : IdList, the building for training\n    bprob = IdProbility, the probabilities for the training builings. \n    activation_prob = ActivationProbility, the probabilities for the activations\n    \n"

In [7]:
train_builing = [14, 19, 28, 39]
bprob = []
collection = {} # training data
activation_prob = {} #  ActivationProbility

In [8]:
# prepare the related config
for item in sorted(train_builing):
    pathfile = os.path.join(path,'multi_group',str(item))
    aprob = []
    ActivationsList = os.listdir(pathfile)
    ActivationCollection = {}

    for activation in ActivationsList:
        activation_data = pd.read_csv(pathfile+'/'+activation, index_col=0)
        #print(len(activation_data))
        aprob.append(len(activation_data))
        ActivationCollection[str(activation[:-15])]=activation_data
    bprob.append(sum(aprob))
    collection['id_'+str(item)] =  ActivationCollection
    activation_prob['id_'+ str(item)] = [i/sum(aprob) for i in aprob]
building_prob = [i/sum(bprob) for i in bprob]

In [9]:
class Sequence(object):
    """
    Attributes
    ----------
    input : np.ndarray
    target : np.ndarray
    all_appliances : pd.DataFrame
        Column names are the appliance names.
    metadata : dict
    weights : np.ndarray or None
    """
    def __init__(self, shape, target_channels_in_list):
        self.input = np.zeros(shape, dtype=np.float32)
        self.target = {}
        for target_channel in target_channels_in_list:
            self.target[str(target_channel)] = np.zeros(shape, dtype=np.float32)
       


In [27]:
# the RealSource, which will replace the original one
class RealSource(object):
    
    def __init__(self, data_in_dictionary, target_channels_in_list, seq_length, IdList, IdProbility, ActivationProbility,
                num_seq_per_batch=32):
        self.data_in_dictionary = data_in_dictionary
        self.target_channels_in_list = target_channels_in_list
        self.seq_length = seq_length
        self.IdList = IdList
        self.IdProbility = IdProbility
        self.ActivationProbility = ActivationProbility
        self.num_seq_per_batch = num_seq_per_batch
    
    def _select_building(self, IdList, IdProbility):
        """
        For Example:
        
        _select_building(train_builing, building_prob), where
        train_builing = [1, 14, 19]
        building_prob = [0.5, 0.2, 0.3]
        """
        return np.random.choice(IdList, 1, p=IdProbility)
    
    def _select_activation(self, ActivationList, ActivationProbility):
        """
        For Example:
        
        _select_activation(range(len(activation_prob['id_1'])), activation_prob['id_1']), where
        range(len(activation_prob['id_1'])) = [0,1,2,3,5]
        activation_prob['id_1'] = [0.1, 0.1, 0.3, 0.2, 0.3]
        """
        return np.random.choice(ActivationList, 1, p=ActivationProbility)
    
    def get_seq_and_check(self, data, IdList, IdProbility, ActivationProbility):
        """
        get a batch of data
        For Example:
        get_seq_and_check(collection, train_builing, building_prob, activation_prob)
        collection = data
        train_builing = [1, 14, 19]
        building_prob = [0.5, 0.2, 0.3]
        activation_prob = {'id_1':[0.1, 0.1, 0.3, 0.2, 0.3], 
                            'id_14':[0.1, 0.1, 0.3, 0.2, 0.3],
                            'id_19':[0.1, 0.1, 0.3, 0.2, 0.3]}
        
        Warning:
        ------------------------------------------------------------------------
            Currently, setting max_iter == 120, the gap within select_start and end is self.seq_length*2 points
            If the gap is self.seq_length points, it will not success. The cause needs to be figured out
            In the prototype stage, using main as target
        """
        success_for_enough_data = False
        max_iter_for_enough_data = 0
        while not success_for_enough_data:
            max_iter_for_enough_data +=1
            select_building = self._select_building(IdList, IdProbility)[0]
            select_building = 'id_'+str(select_building)
            activation_prob_for_the_select_building = ActivationProbility[select_building ]
            select_activation = self._select_activation(range(len(activation_prob_for_the_select_building)), 
                                           activation_prob_for_the_select_building)[0]      
            get_seq_before_check = data[select_building][str(select_activation)]
            # double check that the index is datetime format
            get_seq_before_check.index = pd.to_datetime(get_seq_before_check.index)
            if len(get_seq_before_check)>=self.seq_length or  max_iter_for_enough_data >= 32 :
                 success_for_enough_data = True
      
        success = False
        max_iter = 0
        while not success:
            max_iter +=1
            select_start = get_seq_before_check.sample(n=1).index[0]
            end = select_start + timedelta(seconds = 60*self.seq_length*2) 
            if len(get_seq_before_check[select_start:end])>=self.seq_length or max_iter==120:
                success = True
                get_seq_after_check = get_seq_before_check[select_start:end]
        
        if max_iter==120:
            seq = None

        else:
            del get_seq_before_check
            #seq = Sequence(self.seq_length)
            seq = Sequence(self.seq_length, self.target_channels_in_list)
            seq.input = np.array(get_seq_after_check[self.target_channels_in_list[0]].values[:self.seq_length])
            #for target in self.target_channels_in_list[1:]:
            #    seq.target = np.array(get_seq_after_check[target].values[:self.seq_length])
            for target in self.target_channels_in_list[1:]:
                seq.target[str(target)] = np.array(get_seq_after_check[target].values[:self.seq_length])
        return seq
    
    def _get_sequence(self):
        seq=self.get_seq_and_check(data = self.data_in_dictionary, 
                                   IdList = self.IdList, 
                                   IdProbility = self.IdProbility, 
                                   ActivationProbility = self.ActivationProbility)
        return seq
    
    def get_batch(self):
        """
        Returns
        -------
        A Batch object or None if source iterator has hit a StopIteration.
        """

        input_sequences = []
        target_sequences = {}
        none_happened = False
        for target in self.target_channels_in_list[1:]:
                target_sequences[str(target)] = []

        for i in range(self.num_seq_per_batch):
            seq = self._get_sequence()
            
            if seq is None:
                none_happened = True
            else:
                input_sequences.append(seq.input.reshape(self.seq_length,1))
                #target_sequences.append(seq.target.reshape(self.seq_length,1))
                for target in self.target_channels_in_list[1:]:
                    target_sequences[str(target)].append(seq.target[str(target)].reshape(self.seq_length,1))
                
        if not none_happened:
            input_sequences = np.asarray(input_sequences).reshape(self.num_seq_per_batch,self.seq_length,1)
            #target_sequences = np.asarray(target_sequences).reshape(self.num_seq_per_batch,self.seq_length,1)
            for target in self.target_channels_in_list[1:]:
                target_sequences[str(target)] = np.asarray(target_sequences[str(target)]).reshape(self.num_seq_per_batch,self.seq_length,1)
        else:
            input_sequences = None
            target_sequences = None
            
        return input_sequences, target_sequences

    

In [50]:
class SynSource(RealSource):
    def __init__(self, data_in_dictionary, target_channels_in_list, seq_length, IdList, IdProbility, ActivationProbility,
                num_seq_per_batch=32):
        
        self.data_in_dictionary = data_in_dictionary
        self.target_channels_in_list = target_channels_in_list
        self.seq_length = seq_length
        self.IdList = IdList
        self.IdProbility = IdProbility
        self.ActivationProbility = ActivationProbility
        self.num_seq_per_batch = num_seq_per_batch
        
    def _get_sequence(self):
        seq = Sequence(self.seq_length, self.target_channels_in_list)
   
        for target in self.target_channels_in_list[1:]:
            channel_seq = super(SynSource,self)._get_sequence()
            seq.target[str(target)] = np.array(channel_seq.target[str(target)])
            seq.input += seq.target[str(target)]
            
        return seq
    

            
            

In [51]:
syn = SynSource(data_in_dictionary = collection, 
                  target_channels_in_list = ['main','fridge','air conditioner'], 
                  seq_length=60, 
                  IdList = train_builing, 
                  IdProbility = building_prob,
                  ActivationProbility = activation_prob)
main, target = syn.get_batch()

In [55]:
main[31]

array([[  25.],
       [  25.],
       [1008.],
       [1002.],
       [1064.],
       [1073.],
       [1078.],
       [ 208.],
       [1212.],
       [1212.],
       [1085.],
       [ 912.],
       [ 908.],
       [ 908.],
       [  25.],
       [  25.],
       [  25.],
       [  25.],
       [ 847.],
       [ 883.],
       [ 890.],
       [ 890.],
       [ 899.],
       [  25.],
       [  25.],
       [  25.],
       [ 239.],
       [1002.],
       [1071.],
       [1071.],
       [1083.],
       [1092.],
       [1092.],
       [1089.],
       [1088.],
       [ 918.],
       [ 909.],
       [ 909.],
       [ 906.],
       [ 902.],
       [ 904.],
       [  25.],
       [  25.],
       [  25.],
       [  25.],
       [ 847.],
       [ 889.],
       [ 909.],
       [ 909.],
       [ 916.],
       [1124.],
       [ 210.],
       [ 210.],
       [1269.],
       [1054.],
       [1053.],
       [1103.],
       [1107.],
       [ 920.],
       [ 931.]], dtype=float32)

In [56]:
target['air conditioner'][31]+target['fridge'][31]

array([[  25.],
       [  25.],
       [1008.],
       [1002.],
       [1064.],
       [1073.],
       [1078.],
       [ 208.],
       [1212.],
       [1212.],
       [1085.],
       [ 912.],
       [ 908.],
       [ 908.],
       [  25.],
       [  25.],
       [  25.],
       [  25.],
       [ 847.],
       [ 883.],
       [ 890.],
       [ 890.],
       [ 899.],
       [  25.],
       [  25.],
       [  25.],
       [ 239.],
       [1002.],
       [1071.],
       [1071.],
       [1083.],
       [1092.],
       [1092.],
       [1089.],
       [1088.],
       [ 918.],
       [ 909.],
       [ 909.],
       [ 906.],
       [ 902.],
       [ 904.],
       [  25.],
       [  25.],
       [  25.],
       [  25.],
       [ 847.],
       [ 889.],
       [ 909.],
       [ 909.],
       [ 916.],
       [1124.],
       [ 210.],
       [ 210.],
       [1269.],
       [1054.],
       [1053.],
       [1103.],
       [1107.],
       [ 920.],
       [ 931.]])